# Spam Classifier

## About Dataset 

Build spam classifier using dataset from [Apache SpamAssassin's public datasets](https://spamassassin.apache.org/old/publiccorpus/). Two datasets are used, namely "20030228_easy_ham.tar.bz2" and "20030228_spam.tar.bz2".

In [12]:
import tarfile, os

spam_path = "spam_dataset"
ham_tar = tarfile.open("20030228_easy_ham.tar.bz2", "r:bz2")  
ham_tar.extractall(path=spam_path)
ham_filenames = [name for name in sorted(os.listdir(os.path.join(spam_path, "easy_ham"))) if len(name) > 20]
ham_tar.close()
spam_tar = tarfile.open("20030228_spam.tar.bz2", "r:bz2")  
spam_tar.extractall(path=spam_path)
spam_filenames = [name for name in sorted(os.listdir(os.path.join(spam_path, "spam"))) if len(name) > 20]
spam_tar.close()

In [13]:
len(ham_filenames)

2500

In [15]:
len(spam_filenames)

500

In [16]:
# Use `email` module from python to parse these emails.
import email
import email.policy

def load_email(is_spam, filename, spam_path=spam_path):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as file:
        return email.parser.BytesParser(policy=email.policy.default).parse(file)

In [18]:
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [27]:
# Look at an example of ham emails
print(ham_emails[5].get_content().strip())

> I just had to jump in here as Carbonara is one of my favourites to make and 
> ask 
> what the hell are you supposed to use instead of cream? 

Isn't it just basically a mixture of beaten egg and bacon (or pancetta, 
really)? You mix in the raw egg to the cooked pasta and the heat of the pasta 
cooks the egg. That's my understanding.

Martin

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [26]:
# Look at an example of spam emails
print(spam_emails[5].get_content().strip())

A POWERHOUSE GIFTING PROGRAM You Don't Want To Miss! 
 
  GET IN WITH THE FOUNDERS! 
The MAJOR PLAYERS are on This ONE
For ONCE be where the PlayerS are
This is YOUR Private Invitation

EXPERTS ARE CALLING THIS THE FASTEST WAY 
TO HUGE CASH FLOW EVER CONCEIVED
Leverage $1,000 into $50,000 Over and Over Again

THE QUESTION HERE IS:
YOU EITHER WANT TO BE WEALTHY 
OR YOU DON'T!!!
WHICH ONE ARE YOU?
I am tossing you a financial lifeline and for your sake I 
Hope you GRAB onto it and hold on tight For the Ride of youR life!

Testimonials

Hear what average people are doing their first few days:
�We've received 8,000 in 1 day and we are doing that over and over again!' Q.S. in AL
 �I'm a single mother in FL and I've received 12,000 in the last 4 days.� D. S. in FL
�I was not sure about this when I sent off my $1,000 pledge, but I got back $2,000 the very next day!� L.L. in KY
�I didn't have the money, so I found myself a partner to work this with. We have received $4,000 over the last 2 days